## 1.1 中文文本预处理步骤

### 一. 去除数据中非文本部分  
 例如杀出特殊字符, `<html>`标签等元素
 
### 二. 处理中文编码问题  
1. python中unicode为明文, string为密文, string需要指定编码格式  
2. 从文件中读取出来的是string, jieba的输入输出使用unicode, 停用词词表使用unicode

### 三. 中文分词 
1. jieba分词, 输出Generater, 可以使用for遍历每个被切分出来的单词
2. jieba分词模式   
  1. 精确模式 : 默认模式, 使用与句子切分, 切分出的词之间没有重叠
  2. 全模式 : 把句子切分成所有可能的词, 词之间存在大量存在大量重叠
  3. 搜索模式 : 适用于搜索任务, 把句子切分成可供搜索的词 , 词之间有少量重叠  
3. jieba对某些人名,地名吃力不好, 此时可通过`suggest_freq(segment, tune=True)`, 加入自定义的segment词频, 使其不能被分割
4. jieba默认使用HMM模型发现新词, 进而把它切分出来.若不想产生新词频, 可以设置jieba.cut(str,HMM=False)

In [3]:
import jieba

# 1. 分词模式 
seg_list = jieba.cut('我来到北京清华大学',cut_all=False)
print '精确模式: ','\ '.join(seg_list)
seg_list = jieba.cut('我来到北京清华大学',cut_all=True)
print '全模式: ','\ '.join(seg_list)
seg_list = jieba.cut_for_search('我来到北京清华大学')
print '搜索模式: ','\ '.join(seg_list)

# 2. jieba自定义词频
with open('data/nlp_test0.txt','r') as f:
    doc = f.read()
    jieba.suggest_freq('沙瑞金',True)
    jieba.suggest_freq('易学习',True)
    jieba.suggest_freq('李达康',True)
    jieba.suggest_freq('王大路',True)
    doc_cut = jieba.cut(doc.decode('utf-8'))
    res = ' '.join(doc_cut)
    print type(res) # unicode
    print res
    
# 3. 切分单词
print '\ '.join(jieba.cut('丰田太省了', HMM=True))
print '\ '.join(jieba.cut('丰田太省了', HMM=False))

精确模式:  我\ 来到\ 北京\ 清华大学
全模式:  我\ 来到\ 北京\ 清华\ 清华大学\ 华大\ 大学
搜索模式:  我\ 来到\ 北京\ 清华\ 华大\ 大学\ 清华大学
<type 'unicode'>
沙瑞金 赞叹 易学习 的 胸怀 ， 是 金山 的 百姓 有福 ， 可是 这件 事 对 李达康 的 触动 很大 。 易学习 又 回忆起 他们 三人 分开 的 前一晚 ， 大家 一起 喝酒 话别 ， 易学习 被 降职 到 道口 县当 县长 ， 王大路 下海经商 ， 李达康 连连 赔礼道歉 ， 觉得 对不起 大家 ， 他 最 对不起 的 是 王大路 ， 就 和 易学习 一起 给 王大路 凑 了 5 万块 钱 ， 王大路 自己 东挪西撮 了 5 万块 ， 开始 下海经商 。 没想到 后来 王大路 竟然 做 得 风生水 起 。 沙瑞金 觉得 他们 三人 ， 在 困难 时期 还 能 以沫 相助 ， 很 不 容易 。 

丰田\ 太省\ 了
丰田\ 太\ 省\ 了


### 四. 引入停用词
1. 停用词一般总结成一个文件, 文件的每行都是一个停用词  
 此处给出一个1208个停用词的版本
2. 删除停用词的思路 : 先用jieba分词, 再遍历停用词表, 删除停用词

In [2]:
stpword_path = 'data/stop_words.txt'
# 1. 生成utf-8的停用词表
with open(stpword_path,'rb') as f:
    content = f.read()
    # 按行切分停用词表
    stpword_list = content.splitlines()
    # 转变为unicode
    stpword_list = [word.decode('GBK') for word in stpword_list]
    
# 2. 分词后删除停用词
res = []
with open('data/nlp_test0.txt','r') as f:
    content = f.read().decode('utf-8')
    generator = jieba.cut(content)
    i = 0
    for word in generator:
        i = i+1
        if word not in stpword_list and word != '\t':
            res.append(word.encode('utf-8'))
print 'before clear stop word: ',i
print 'after clear stop word: ',len(res)
print ' '.join(res)

before clear stop word:  110
after clear stop word:  57
沙瑞金 赞叹 易学习 胸怀 金山 百姓 有福 这件 事 李达康 触动 很大 易学习 回忆起 三人 分开 前一晚 一起 喝酒 话别 易学习 降职 道口 县当 县长 王大路 下海经商 李达康 赔礼道歉 觉得 对不起 最 对不起 王大路 易学习 一起 王大路 凑 万块 钱 王大路 东挪西撮 万块 下海经商 没想到 王大路 做 风生水 沙瑞金 觉得 三人 困难 时期 以沫 相助 容易 




## 1.2 TF-IDF文档表示

### 一. 何为TF-IDF:  
1. TF-IDF-词频逆文档频率 (Term Frequency inverse Document Frequency)
2. TF-IDF把文章表示成一个向量.该向量的每个元素表示对应文章中该单词的出现次数(词频).如果单词表有10,000个单词, 则向量有10,000个元素,每个元素=词频*权重
3. TF-IDF是突出重要单词的一种文档表示方法. 因为像"the of"这样的单词, 在文章中会大量出现.如果直接用其词频放在向量中, 则这样的单词会对最终计算k临近距离产生较大影响.因此向量中每个元素使用词频*权重表示.何为重要单词, 需要满足以下两个条件:   
  1. 在本文章中大量出现(词频)  
  2. 在语料库中的其他文章很少出现(词频) 
  
### 二. 强调重点单词 emphasize important words
1. 局部地方频繁出现 - appear frequently in document (common locally)   
 TF : Term Frequency词频 = {x,x,x ... x}(10,000个单词,每个单词出现的次数)
2. 语料库中很少出现 - rarely apperance in corpus (rarely global)  
 IDF : Inverse doc frequence逆文档频率 = $\log { \frac { \#docs }{ 1+\#doc\quad using\quad word }  } = \log { \frac { 语料库总文档数 }{ 1+使用该单词的文档数 }  }  $
3. 文档表示向量 = $TF*IDF$ (词频*权重)